# Dataset Creation

Before we can create a LoRA checkpoint, we need a decent dataset to train on.

I found this: https://github.com/DeepSoftwareAnalytics/CommitMsgEmpirical

However, it does need a bit of cleanup

In [1]:
import tarfile
import pickle
import sqlite3
import os
import pandas as pd

## Step 1. Download and extract the dataset

In [ ]:
# Download the data
# !curl https://zenodo.org/records/5025758/files/raw_data.tar.gz?download=1 --output raw_data.tar.gz

In [2]:
# FILENAME = 'raw_data.tar.gz'
ROOT_DIR = 'dataset'

In [3]:
# # Extract the .tar.gz file
# tar = tarfile.open(FILENAME, "r:gz")
# tar.extractall()
# tar.close()

## Step 2. Create a Sqlite database

In [4]:
# # Create the sqlite database
# !rm commits.db
# conn = sqlite3.connect('commits.db')
# c = conn.cursor()

In [5]:
# # Create the tables
# queries = []
# queries.append("""
# CREATE TABLE Organizations (
#     organization_id INTEGER PRIMARY KEY AUTOINCREMENT,
#     organization_name TEXT NOT NULL
# );
# """)

# queries.append("""
# CREATE TABLE Repositories (
#     repo_id INTEGER PRIMARY KEY AUTOINCREMENT,
#     repo_name TEXT NOT NULL,
#     organization_id INTEGER,
#     FOREIGN KEY (organization_id) REFERENCES Organizations (organization_id)
# );
# """)

# queries.append("""
# CREATE TABLE Languages (
#     language_id INTEGER PRIMARY KEY AUTOINCREMENT,
#     language_name TEXT NOT NULL
# );
# """)

# queries.append("""
# CREATE TABLE Commits (
#     commit_id INTEGER PRIMARY KEY AUTOINCREMENT,
#     repo_id INTEGER NOT NULL,
#     language_id INTEGER,
#     commit_message TEXT NOT NULL,
#     git_diff TEXT NOT NULL,
#     FOREIGN KEY (repo_id) REFERENCES Repositories (repo_id),
#     FOREIGN KEY (language_id) REFERENCES Languages (language_id)
# );
# """)

# for query in queries:
#     c.execute(query)

# conn.commit()

In [6]:
# for dirpath, dirnames, filenames in os.walk(ROOT_DIR):
#     for file in filenames:
#         try:
#             _, lang, org = dirpath.split('/')
#             # print(lang, org, file)

#             # Add the language and organization to the database
#             c.execute("INSERT OR IGNORE INTO Languages (language_name) VALUES (?)", (lang,))
#             c.execute("INSERT OR IGNORE INTO Organizations (organization_name) VALUES (?)", (org,))
#             conn.commit()

#             # Add the repo
#             lang_fk = c.execute("SELECT language_id FROM Languages WHERE language_name = ?", (lang,)).fetchone()[0]
#             org_fk = c.execute("SELECT organization_id FROM Organizations WHERE organization_name = ?", (org,)).fetchone()[0]
#             c.execute("INSERT OR IGNORE INTO Repositories (repo_name, organization_id) VALUES (?, ?)", (file, org_fk))
#             conn.commit()

#             # Time to read the commit data
#             with open(os.path.join(dirpath, file), 'rb') as f:
#                 commits = pickle.load(f)

#             for commit in commits:
#                 commit_message = commit['msg']
#                 git_diff = commit['diff']
#                 # Use parameterized queries to insert commit data
#                 c.execute("INSERT INTO Commits (repo_id, language_id, commit_message, git_diff) VALUES (?, ?, ?, ?)", (org_fk, lang_fk, commit_message, git_diff))
#                 conn.commit()
#         except pickle.UnpicklingError as e:
#             print(f"Failed to unpickle file: {file}, error: {e}")
#         except Exception as e:
#             print(f"An error occurred: {e}")

## Step 3. Review the output

In [7]:
# # Get list of tables
# c.execute("SELECT name FROM sqlite_master WHERE type='table';")
# # tables = c.fetchall()
# print("Tables in the database:", tables)

In [8]:
# # Choose a table to describe
# tables = ['Organizations', 'Repositories', 'Languages', 'Commits']

# # Get the table information (schema)
# for table in tables:
#     c.execute(f"PRAGMA table_info({table});")
#     table_info = c.fetchall()
#     print(f"Schema of {table}:", table_info)

In [9]:
conn = sqlite3.connect('commits.db')
df = pd.read_sql_query("SELECT * FROM Commits LIMIT 100;", conn)
df.head()

,commit_id,repo_id,language_id,commit_message,git_diff
0,1,2,1,Merge pull request #1716 from sujit510/feature...,diff --git a/snippets/isAlpha.md b/snippets/is...
1,2,2,1,Update and rename isAlphabetOnly.md to isAlpha.md,diff --git a/snippets/isAlpha.md b/snippets/is...
2,3,2,1,Add substring count/indexes,diff --git a/snippets/countSubstrings.md b/sni...
3,4,2,1,Add generatorToArray,diff --git a/snippets/generatorToArray.md b/sn...
4,5,2,1,Add storage testers,diff --git a/snippets/isLocalStorageEnabled.md...


In [11]:
df.shape

(100, 5)

In [12]:
conn.close()

In [13]:
conn = sqlite3.connect('commits.db')
cursor = conn.cursor()

# Get list of tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:", tables)

# Choose a table to describe
table_to_describe = 'Commits'

# Get the table information (schema)
cursor.execute(f"PRAGMA table_info({table_to_describe});")
table_info = cursor.fetchall()
print(f"Schema of {table_to_describe}:", table_info)

# Get a sample of data from the table
cursor.execute(f"SELECT * FROM {table_to_describe} LIMIT 10;")
sample_data = cursor.fetchall()
print(f"Sample data from {table_to_describe}:", sample_data)

# Count the number of rows in the table
cursor.execute(f"SELECT COUNT(*) FROM {table_to_describe};")
row_count = cursor.fetchone()
print(f"Total number of rows in {table_to_describe}:", row_count[0])

# Close the connection
conn.close()

Tables in the database: [('Organizations',), ('sqlite_sequence',), ('Repositories',), ('Languages',), ('Commits',)]
Schema of Commits: [(0, 'commit_id', 'INTEGER', 0, None, 1), (1, 'repo_id', 'INTEGER', 1, None, 0), (2, 'language_id', 'INTEGER', 0, None, 0), (3, 'commit_message', 'TEXT', 1, None, 0), (4, 'git_diff', 'TEXT', 1, None, 0)]
Sample data from Commits: [(1, 2, 1, 'Merge pull request #1716 from sujit510/feature/isAlphabetOnly\n\nAdded snippet for isAlphabetOnly', "diff --git a/snippets/isAlpha.md b/snippets/isAlpha.md\nnew file mode 100644\nindex 000000000..5a823b506\n--- /dev/null\n+++ b/snippets/isAlpha.md\n@@ -0,0 +1,18 @@\n+---\n+title: isAlpha\n+tags: string,regexp,beginner\n+---\n+\n+Checks if a string contains only alpha characters.\n+\n+- Use `RegExp.prototype.test()` to check if the given string matches against the alphabetic regexp pattern.\n+\n+```js\n+const isAlpha = str => /^[a-zA-Z]*$/.test(str);\n+```\n+\n+```js\n+isAlpha('sampleInput'); // true\n+isAlpha('this 